In [1]:
!nvidia-smi

Wed Aug 16 12:06:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install git+https://github.com/bofenghuang/vigogne

  Cloning https://github.com/bofenghuang/vigogne to /tmp/pip-req-build-gc1v_6ic
  Running command git clone --filter=blob:none --quiet https://github.com/bofenghuang/vigogne /tmp/pip-req-build-gc1v_6ic
  Resolved https://github.com/bofenghuang/vigogne to commit 1911436ba3da456de7d63e5e830ac8bf5b88ac6c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-install-6y62qhdi/accelerate_714879a43de045baa75e4a5f012e7448
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-install-6y62qhdi/accelerate_714879a43de045baa75e4a5f012e7448
  Resolved https://github.com/huggingface/accelerate.git to commit 64c586f5eb52c86f045b1d46202c60fbbdbdd9c9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (p

In [ ]:
# Uncomment if necessary
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, TextStreamer
from vigogne.preprocess import generate_inference_chat_prompt

In [4]:
base_model_name_or_path = "bofenghuang/vigogne-2-7b-chat"

tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path, padding_side="right", use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    torch_dtype=torch.float16,
    device_map="auto",
    # load_in_8bit=True,
    # trust_remote_code=True,
    # low_cpu_mem_usage=True,
)

# lora_model_name_or_path = ""
# model = PeftModel.from_pretrained(model, lora_model_name_or_path)

model.eval()

if torch.__version__ >= "2":
    model = torch.compile(model)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [8]:
streamer = TextStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)

def infer(
    user_query,
    temperature=0.1,
    top_p=1.0,
    top_k=0,
    max_new_tokens=512,
    **kwargs,
):
    prompt = generate_inference_chat_prompt(user_query, tokenizer=tokenizer)
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(model.device)
    input_length = input_ids.shape[1]

    generated_outputs = model.generate(
        input_ids=input_ids,
        generation_config=GenerationConfig(
            temperature=temperature,
            do_sample=temperature > 0.0,
            top_p=top_p,
            top_k=top_k,
            max_new_tokens=max_new_tokens,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            **kwargs,
          ),
        streamer=streamer,
        return_dict_in_generate=True,
    )
    generated_tokens = generated_outputs.sequences[0, input_length:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return generated_text

In [15]:
def chat(**kwargs):
    history = []
    while True:
        user_input = input(">> <|user|>: ")
        print(">> <|assistant|>: ", end="")
        model_response = infer([*history, [user_input, ""]], **kwargs)

        history.append([user_input, model_response])
        # print(f">> <|assistant|>: {history[-1][1]}")

In [16]:
chat()

>> <|user|>: Expliquez la différence entre DoS et phishing.
>> <|assistant|>: DoS (Denial of Service) est une attaque qui consiste à bloquer l'accès à un service ou à un site web en surchargeant le serveur avec des requêtes. Phishing est une technique de fraude qui consiste à envoyer des messages frauduleux pour obtenir des informations personnelles ou des informations de paiement.
>> <|user|>: Et DDoS ?
>> <|assistant|>: DDoS (Distributed Denial of Service) est une attaque de déni de service distribué qui consiste à cibler un serveur ou un réseau en envoyant une quantité excessive de requêtes. Cette attaque peut être menée par un grand nombre d'ordinateurs infectés par des logiciels malveillants ou par des machines automatiques appelées "botnets". Les attaquants peuvent utiliser des techniques de vol de données, de vol d'identité ou de vol de crédit-carte pour obtenir des informations personnelles ou financières. Les attaques DDoS peuvent causer des perturbations importantes pour les 

KeyboardInterrupt: ignored